## Data origin :: smard

The origin of this data is the transparency "smard" from "Bundesnetzagentur". It's a central collection and publiaction of electricity generation, transportation and consumption data and information of the german electricty market. The url for the following data is: [smard; Data](https://www.smard.de/home/downloadcenter/download-marktdaten#!?downloadAttributes=%7B%22selectedCategory%22:false,%22selectedSubCategory%22:false,%22selectedRegion%22:false,%22from%22:1628373600000,%22to%22:1629323999999,%22selectedFileType%22:false%7D)


## Feature Description

Features are documented in the [smard manual](https://www.smard.de/home/benutzerhandbuch).

## Import

In [1]:
import datetime, time, os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pandas as pd
import glob
#from pandas_profiling import ProfileReport
import json

#from cesium import datasets

from functools import reduce

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Make numpy printouts easier to read.
#np.set_printoptions(precision=3, suppress=True)

from datetime import datetime, timezone, timedelta
import pytz
resample_factor = 15

import warnings

### Time Zone Adaption

All time information on SMARD refers to the time valid at the respective time
Central European Standard Time (CET or UTC+1) or Central European Summer Time (CEST or UTC+2).
(CEST or UTC+2).
- The changeover from standard time to daylight saving time takes place every last Sunday
in March at 02:00 by skipping the hour from 02:00 to 03:00.
- The changeover from summer time to standard time takes place every last Sunday in
in October at 03:00 by repeating the hour from 02:00 to 03:00
o'clock

## Part 1: Data Mining

## 1.1 Import Forcasted Energy Consumption

In [25]:
df_prog_cons = pd.read_csv("temp_df_prog_cons.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'temp_df_prog_cons.csv'

In [3]:
# Read csv file and assign to dataframe
df_prog_cons = pd.read_csv("data/smard/df_prog_cons.csv")

# Convert time to datetime object and set it as index
df_prog_cons["dt_start_cet"] = pd.to_datetime(df_prog_cons["dt_start_cet"])
df_prog_cons.set_index('dt_start_cet', inplace=True)

# Convert CET time zone to UTC time zone and rename index
df_prog_cons.index = df_prog_cons.index.tz_localize('Europe/Berlin', ambiguous="NaT" , nonexistent="NaT")
df_prog_cons.index = df_prog_cons.index.tz_convert(pytz.utc)
df_prog_cons.index.names = ['dt_start_utc']
df_prog_cons.index = pd.to_datetime(df_prog_cons.index)
df_prog_cons.index = df_prog_cons.index.tz_localize(None)

df_prog_cons.to_csv("temp_prg_cons.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'data/smard/df_prog_cons.csv'

In [4]:
df_prog_cons.tail()

NameError: name 'df_prog_cons' is not defined

In [5]:
df_prog_cons['total_pred_cons'] = df_prog_cons.sum(axis=1)

NameError: name 'df_prog_cons' is not defined

In [6]:
count = np.isinf(df_prog_cons).values.sum()
print("The data frame contains " + str(count) + " infinite values and",df_prog_cons.isnull().sum().sum(),"missing values.")

NameError: name 'df_prog_cons' is not defined

## 1.3 Realisierte Erzeugung

In [7]:
df = pd.read_csv("data/smard/Stromerzeugung/Realisierte_Erzeugung_2021.csv", delimiter=";", decimal=',')

df["datetime"]= df['Datum'] + ' ' + df['Uhrzeit']
df.drop(columns=['Datum', 'Uhrzeit'], inplace=True)
df["datetime"] = pd.to_datetime(df["datetime"], dayfirst=True)

#df_prog_cons["dt_start_utc"] = df_prog_cons["dt_start_utc"] - timedelta(hours=1)


df["datetime"] = pd.to_datetime(df["datetime"])
df.set_index('datetime', inplace=True)

df.index = df.index.tz_localize('Europe/Berlin', ambiguous="NaT" , nonexistent="NaT")
df.index = df.index.tz_convert(pytz.utc)
df.index = pd.to_datetime(df.index)
df.index = df.index.tz_localize(None)






FileNotFoundError: [Errno 2] No such file or directory: 'data/smard/Stromerzeugung/Realisierte_Erzeugung_2021.csv'

In [8]:
df.head()

NameError: name 'df' is not defined

In [9]:
df.index.names = ['dt_start_utc']
df.head()

NameError: name 'df' is not defined

In [10]:
#cols = df.columns.tolist()
#cols =cols.remove("datetime")

In [11]:
#print(cols)

In [12]:
df["Erdgas[MWh]"] = df["Erdgas[MWh]"].replace("-", int(0))
#df["Erdgas[MWh]"] = df["Erdgas[MWh]"].replace(".", "")
#df["Erdgas[MWh]"] = df["Erdgas[MWh]"].astype('int64')

df = df.astype("float32", copy=False)


df


df['rel_total'] = df.sum(axis=1)
df.dtypes
df.head()

NameError: name 'df' is not defined

Publishing the data is about 2 hours delayed. We need to shif them 15 min * 8

In [13]:
df = df.shift(periods=8)

NameError: name 'df' is not defined

In [14]:
df_real_prov = df.copy()

NameError: name 'df' is not defined

In [15]:
df_real_prov.head()

NameError: name 'df_real_prov' is not defined

In [16]:
df.to_csv("test_erdgas.csv")

NameError: name 'df' is not defined

## Merge

In [26]:
df_merge = df_prog_cons.merge(df_real_prov, left_index=True, right_index=True)

NameError: name 'df_prog_cons' is not defined

In [27]:
df_merge.head()

NameError: name 'df_merge' is not defined

In [28]:
df_merge.eval("diff_prog_real = total_pred_cons - rel_total", inplace=True)

NameError: name 'df_merge' is not defined

In [29]:
df_merge.tail()

NameError: name 'df_merge' is not defined

## Save Data Set

In [30]:
df_merge.to_pickle('data/pickle/df_merged_1.pickle')

NameError: name 'df_merge' is not defined

In [31]:
count = np.isinf(df_prog_cons).values.sum()
print("The data frame contains " + str(count) + " infinite values")
print("The Data Frame has",df_prog_cons.isnull().sum().sum(),"missing values.")

NameError: name 'df_prog_cons' is not defined

In [32]:
#df.fillna(0, inplace=True)
#df.fillna(df.median(), inplace=True)